In [2]:
import sys
sys.path.append("..")
from SINC_functions import *
import matplotlib.pyplot as plt

In [57]:
import numpy as np
import scipy.optimize
from numpy import linalg as la
from scipy.special import gammaln as lgamma
from scipy.special import digamma as dgamma
from multiprocessing import Pool
import time
# import multiprocessing
import warnings
warnings.filterwarnings("ignore")


def E_step(omega, v0, v1, theta):
    temp_dens = np.zeros((P, P, 2));

    dens1 = - 0.5 * np.log(v1 * v1 * 2 * np.pi) - (omega ** 2) / (2 * v1 * v1) + np.log(theta);
    temp_dens[:, :, 1] = dens1;
    dens0 = - 0.5 * np.log(v0 * v0 * 2 * np.pi) - (omega ** 2) / (2 * v0 * v0) + np.log(1 - theta);
    temp_dens[:, :, 0] = dens0;
    abmax = np.amax(temp_dens, axis=2);
    EZ_new = np.exp(dens1 - abmax) / (np.exp(dens0 - abmax) + np.exp(dens1 - abmax));
    np.fill_diagonal(EZ_new, 0)

    Ed_new = EZ_new / v1 / v1 + (1 - EZ_new) / v0 / v0;
    return (EZ_new, Ed_new)

def M_step_prec(omega, Ed, S, lamb):
    out = omega
    pseq = range(P)
    for p in range(P):
        remove_i = np.delete(pseq, p);
        out_mi_mi = out[remove_i];
        out_mi_mi = out_mi_mi[:, remove_i];
        Ed_i_mi = Ed[remove_i];
        Ed_i_mi = Ed_i_mi[:, p]
        Ed_i_mi = np.diag(Ed_i_mi);
        S_i_mi = S[remove_i];
        S_i_mi = S_i_mi[:, p]
        v = N / (lamb + S[p, p]);
        invsub = la.inv(out_mi_mi);
        u = -np.matmul(la.inv((lamb + S[p, p]) * invsub + Ed_i_mi), S_i_mi)
        out[p, remove_i] = u.T;
        out[remove_i, p] = u;
        univu = np.matmul(np.matmul(u.T, invsub), u);
        out[p, p] = v + univu;

    return ([out, omega])

def M_step_theta(EZ, a, b):
    EZ_sum = np.sum(EZ, axis=(0, 1)) / 2
    PP = (P * (P - 1)) / 2
    theta = (a - 1 + EZ_sum) / (a + b + PP - 2);
    return (theta)


def s(x):
    return (sum(x))

def TPR_FPR(adj_true, adj_est):
    from sklearn.metrics import confusion_matrix
    import numpy as np

    ind = np.tril_indices(P, -1)

    tn, fp, fn, tp = (confusion_matrix(adj_true[ind], adj_est[ind]).ravel()) * 1.0
    np = (np.sum(adj_true[ind])) * 1.00
    nn = sum((adj_true[ind] == 0) * 1) * 1.00
    TPR = tp / np
    FPR = fp / (fp + tn)
    return ([TPR, FPR])


def F1(adj_true, adj_est):
    from sklearn.metrics import confusion_matrix
    import numpy as np

    ind = np.tril_indices(P, -1)

    tn, fp, fn, tp = (confusion_matrix(adj_true[ind], adj_est[ind]).ravel()) * 1.0
    prec = tp / (tp + fp)
    recall = tp / (tp + fn)
    F1 = 2 * (prec * recall) / (prec + recall)
    MCC = ((tp * tn) - (fp * fn)) / (np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)))
    return ([F1, MCC])

def Performance_B(B_true,B_est):
    TP = np.sum((B_true == 1) & (B_est == 1)) * 1.0
    FP = np.sum((B_true == 0) & (B_est == 1)) * 1.0
    FN = np.sum((B_true == 1) & (B_est == 0)) * 1.0
    TN = np.sum((B_true == 0) & (B_est == 0)) * 1.0
    NN = np.sum((B_true == 0)) * 1.0
    NP = np.sum((B_true == 1)) * 1.0
    
    prec = TP / (TP + FP)
    recall = TP / (TP + FN)
    
    F1 = 2*(prec*recall) / (prec + recall)
    MCC = ((TP*TN) - (FP*FN)) / np.sqrt((TP + FP)*(TP + FN)*(TN + FP)*(TN + FN))
    
    TPR = TP / (NP)
    FPR = FP / (NN)
    return([TPR,FPR,MCC,F1])

def Performance_Omega(adj_true,adj_est):
    TP = np.sum((adj_true == 1) & (adj_est == 1)) / 2.0
    FP = np.sum((adj_true == 0) & (adj_est == 1)) / 2.0
    FN = np.sum((adj_true == 1) & (adj_est == 0)) / 2.0
    TN = np.sum((adj_true == 0) & (adj_est == 0)) / 2.0
    NN = np.sum((adj_true == 0)) / 2.0
    NP = np.sum((adj_true == 1)) / 2.0
    
    prec = TP / (TP + FP)
    recall = TP / (TP + FN)
    
    F1 = 2*(prec*recall) / (prec + recall)
    MCC = ((TP*TN) - (FP*FN)) / np.sqrt((TP + FP)*(TP + FN)*(TN + FP)*(TN + FN))
    
    TPR = TP / (NP)
    FPR = FP / (NN)
    return([TPR,FPR,MCC,F1])

def objective_z(z, *args):

    x = args[1]
    mu_b = args[2]
    prec = args[3]
    alpha = np.exp(z)
    A = z - mu_b
    A = np.matrix(A)
    re = - np.sum(lgamma(alpha + x) - lgamma(alpha)) + (lgamma(np.sum(alpha + x)) - lgamma(np.sum(alpha))) + np.matmul(
        np.matmul(A, prec), A.T)
    return (float(re))


def gradient_z(z, *args):
    x = args[1]
    mu_b = args[2]
    prec = args[3]
    alpha = np.exp(z)
    A = z - mu_b
    A = np.matrix(A)
    re = - ((dgamma(alpha + x) - dgamma(alpha)) + (dgamma(np.sum(alpha + x)) - dgamma(np.sum(alpha)))) * alpha + prec.dot(A.T).T
    return (re)


def M_step_Z(alpha, x, mu, mu_b, prec):
    Z_est = np.zeros((N, P))
    mybounds = []
    for i in range(P):
        mybounds.append((None, None))
    for i in range(N):
        init = Z[i,]
        arg = (alpha[i,], x[i,], mu, mu_b[i,], prec, i)
        z_est = scipy.optimize.fmin_l_bfgs_b(objective_z, fprime=gradient_z, x0=init, args=arg, approx_grad=1,
                                             bounds=mybounds)
        Z_est[i,] = z_est[0]
    return (Z_est)


def inverse_logit(x):
    return(np.exp(x) / (1 + np.exp(x)))     



def M_step_Z_parallel(args):
    mybounds = []
    for i in range(P):
        mybounds.append((None, None))
    alpha, x, mu_b, prec, init = args
    arg = (alpha, x, mu_b, prec)
    z_est = scipy.optimize.fmin_l_bfgs_b(objective_z, fprime=gradient_z, x0=init, args=arg, approx_grad=1,
                                         bounds=mybounds)
    return (z_est[0])


def ELBO_B(X,Y,mu,phi,sig_y,sig_b,v1,theta):
    XtX = -X.T.dot(X)
    elbo = -0.5*(np.linalg.norm(Y - X.dot(mu*phi),2))
    phi = np.maximum(phi,np.zeros(Q) + 0.0000000001)
    for j in range(Q):
        elbo += -(0.5/sig_y)*(XtX[j,j])*(phi[j]*(sig_b[j] + mu[j]**2)*(phi[j]*mu[j])**2) \
            - phi[j]*np.log(phi[j]/theta) -(1 - phi[j])*np.log((1 - phi[j])/(1 - theta)) \
            + (phi[j]/2) * (1 + np.log(sig_b[j]/(v1*sig_y)) - ((sig_b[j] + mu[j]**2)/(v1*sig_y)))
    return(elbo)

def ELBO_omega(Z,M,omega,B,B0,Ed,Ez,lam,pi,N,P,a,b):
    X = Z - (B0 + M.dot(B.T))
    omega2Ed = (omega**2)*Ed
    diag_sum = np.diag(omega2Ed).sum()
    piP = (pi/(1 - pi))*Ez
    piP_diag_sum = np.diag(piP).sum()
    
    elbo = 0.5*N*np.log(np.linalg.det(omega)) - 0.5*np.trace(X.T.dot(X).dot(omega))  \
            -0.25*(omega2Ed.sum() - diag_sum) - 0.5*lam*np.diag(omega).sum() \
            +0.25*(np.log(piP.sum() - piP_diag_sum)) + P*(P-1)/2*np.log(1 - pi) \
            +(a - 1)*np.log(pi) + (b - 1)*np.log(1 - pi)
    return(elbo)

def ELBO_omega(Z,M,omega,B,B0,Ed,Ez,lam,pi,N,P,a,b):
    X = Z - (B0 + M.dot(B.T))
    omega2Ed = (omega**2)*Ed
    diag_sum = np.diag(omega2Ed).sum()
    piP = (pi/(1 - pi))*Ez
    piP_diag_sum = np.diag(piP).sum()
    
    elbo = 0.5*N*np.log(np.linalg.det(omega)) \
            -0.25*(omega2Ed.sum() - diag_sum) - 0.5*lam*np.diag(omega).sum() \
            +0.25*(np.log(piP.sum() - piP_diag_sum)) + P*(P-1)/2*np.log(1 - pi) \
            +(a - 1)*np.log(pi) + (b - 1)*np.log(1 - pi)
    return(elbo)

def ELBO_Z(z, *args):
    
    args = args[0]
    x = args[1]
    mu_b = args[2]
    prec = args[3]
    alpha = np.exp(z)
    A = z - mu_b
    A = np.matrix(A)
    re = - np.sum(lgamma(alpha + x) - lgamma(alpha)) + (lgamma(np.sum(alpha + x)) - lgamma(np.sum(alpha))) + np.matmul(
        np.matmul(A, prec), A.T)
    return (float(re))

def M_step_blocks(omega, Ed, a, b, groups):
    n_groups = len(np.unique(groups)) + 1
    tau_small = np.ones((n_groups, n_groups)) * b
    size = np.zeros((n_groups, n_groups))
    for i in range(P):
        for j in range(P):
            if i < j:
                tau_small[groups[i], groups[j]] = tau_small[groups[i], groups[j]] + (
                0.5 * omega[i, j] * omega[i, j] * Ed[i, j]);
                size[groups[i], groups[j]] = size[groups[i], groups[j]] + 0.5;
                if groups[i] != groups[j]:
                    tau_small[groups[j], groups[i]] = tau_small[groups[j], groups[i]] + (
                    0.5 * omega[j, i] * omega[j, i] * Ed[j, i]);
                    size[groups[j], groups[i]] = size[groups[j], groups[i]] + 0.5;
    tau_small = (a - 1 + size) / tau_small
    tau = np.zeros((P, P))
    for i in range(P):
        for j in range(P):
            tau[i, j] = tau_small[groups[i], groups[j]]
    return (tau)

def VI_VS_parallel(args):
    Y, X, v1, sig_hat, mu, phi, theta, a_gamma, b_gamma = args
    
    sig = np.ones((Q,1)) / 2500
#     theta = 0.01
    xtx = X.T.dot(X)

    mu_change = 100
    while mu_change > .01:
        mu_old = np.copy(mu)
        for j in range(Q):
            X_temp = np.copy(X)
            X_l = np.delete(X_temp,j,axis = 1)
            X_j = X_temp[:,j]
            XtX = xtx[:,j]
            XtX = np.delete(XtX,j,axis = 0)
            phi_temp = np.copy(phi)
            phi_l = np.delete(phi_temp,j,axis = 0)
            mu_temp = np.copy(mu)
            mu_l = np.delete(mu_temp,j,axis = 0)

            mu[j] = (sig[j]/sig_hat) * (X.T.dot(Y)[j] - np.sum(XtX.dot(phi_l * mu_l)))
            sig[j] = sig_hat / (xtx[j,j] + 1/v1)
            phi_j = np.log(theta/(1 - theta)) + np.log(sig[j]/(v1*sig_hat)) + mu[j]**2/(2*sig[j])
            phi[j] = inverse_logit(phi_j)
            if phi_j > 709:
                phi[j] = 1.0

        mu_change = np.max(abs(mu_old - mu))
        theta = (np.sum(phi) + a_gamma - 1) / (Q + a_gamma + b_gamma - 2)
    mu = np.squeeze(mu)
    PHI = np.squeeze(phi)
    elbo_B = ELBO_B(X,Y,mu,phi - 0.00001,sig_hat,sig,v1,theta)
#     print(mu.shape)
    return(mu,np.maximum(PHI,np.zeros(Q)),np.repeat(elbo_B,Q))


def SINC(x, m, v0, v1, lamb, vB,a_gamma,b_gamma,a_pi,b_pi, max_iters, tol_prec, tol_elbo, cpus):
    #######
    # x are OTU measurements where rows are the samples
    # M are the environmental factors
    # v0 is the v0 value in the precision prior
    # v1 is the v1 value in the precision prior
    # vB is the vB value in the B coefficient prior
    # max_iters
    # tol_prec is how much change precision matrix can change each iteration before stopping
    # tol_B is how much change B matrix can change each iteration before stopping
    # lamb is the prior hyperparameter on EMGS part
    ########

    # get dimensions
    global P, N, Q
    P = x.shape[1]
    N = x.shape[0]
    Q = m.shape[1]

    ## initialize values
    global Z
    Z = np.log(x + 1)
    B0 = np.mean(Z,axis = 0)
    alpha = np.exp(Z)
    B = np.zeros((P, Q))
    phi = np.ones((P,Q)) / 4
    mu_b = m.dot(B.T)
    centr = np.mean(Z, axis=0)
    S = ((Z - centr).T).dot((Z - centr))
    sig = (S + np.eye(P)*.1) / N
    omega = la.inv(sig)
    sigs_j = np.zeros(P)
    elbo_score = np.array([])
    theta = np.ones(P) * (a_gamma / (a_gamma + b_gamma))

    ## v0 values
    pii = .5
    
    change_z = 10000
    change_prec = 10000
    change_B = 10000

    iters_total = 0
    
    elbo_change = 10000
    elbo = 0
    tau = np.ones((P,P))
    groups = [0]*P

#     while (change_B > tol_B or change_prec > tol_prec):
    while (elbo_change > tol_elbo):
        if (iters_total >= max_iters):
            break
        iters_total += 1
        
        ## get variance of each row
        pseq = range(P)
        SIG = np.linalg.inv(omega)
        for p in range(P):
            remove_i = np.delete(pseq,p)
            r11 = SIG[remove_i]
            r11 = r11[:,remove_i]
            r12 = SIG[p,remove_i]
            sigs_j[p] = SIG[p,p] - r12.dot(np.linalg.inv(r11)).dot(r12)

        #### update B with VI
        B_old = np.copy(B)            
        pool = Pool(cpus)
        args = [(Z[:,i] - B0[i],m,vB,sigs_j[i],B[i,],phi[i,],theta[i],a_gamma,b_gamma) for i in range(P)]
        VI_update = np.asarray(pool.map(VI_VS_parallel,args))
        B = np.squeeze(VI_update[:,0,:])
        phi = np.maximum(np.zeros((P,Q)),np.squeeze(VI_update[:,1,:]))
        elbo_B = VI_update[:,2,0].sum()
        pool.close()
        pool.join

        B_mult = B * phi
        change_B = np.max(np.absolute(B_old - B))
        mu_b = m.dot(B_mult.T)

        ## E Step
        EZ, Ed = E_step(omega, v0, v1, pii)
        Ed = Ed / tau
       
        ## M Step

        for p in range(P):
            B0[p] = np.mean(Z[:,p] - mu_b[:,p])
            mu_b[:,p] = mu_b[:,p] + B0[p]

        ## update centr
        S = ((Z - mu_b).T).dot((Z - mu_b))

        ## update prec
        change_p = 1
        prec_old = np.copy(omega)
        prec_old_loop = np.copy(omega)
        change_prec = 10000
        while(change_prec > tol_prec):
            out, omega = M_step_prec(omega, Ed, S, lamb)
            pii = M_step_theta(EZ, a_pi, b_pi)
            change_prec = np.amax(np.absolute(prec_old_loop - omega))
            prec_old_loop = np.copy(omega)
            
        theta = (phi.sum(axis = 1) + a_gamma - 1)/(Q + a_gamma + b_gamma - 2)
        
        ## update Z
#         alpha = np.exp(Z)
        pool = Pool(cpus)
        Z_old = np.copy(Z)
        args = [(alpha[i,], x[i,], mu_b[i,],omega, Z[i,]) for i in range(N)]
        Z = np.asarray(pool.map(M_step_Z_parallel, args))
        change_z = np.amax(np.absolute(Z_old - Z))
        pool.close()
        pool.join
#         args = [(alpha[i,], x[i,], mu_b[i,],omega, Z[i,]) for i in range(N)]
        args = [(alpha[i,], x[i,], mu_b[i,],omega) for i in range(N)]
        elbo_Z = 0
        for n in range(N):
            elbo_Z += -ELBO_Z(Z[n,],args[n])
#             elbo_z = M_step_Z_parallel(args[n])
#             elbo_Z += elbo_z.sum()
        
        elbo_omega = ELBO_omega(Z,m,omega,B,B0,Ed,EZ,lamb,pii,N,P,a_pi,b_pi)
        elbo_old = np.copy(elbo)
        elbo = elbo_B + elbo_omega + (elbo_Z/N)
#         elbo_change = abs(elbo_old - elbo)
        if iters_total > 1:
            elbo_change = elbo - elbo_old
        print(elbo_B,elbo_omega,elbo_Z)
        print("ELBO:",elbo_B + elbo_omega + elbo_Z/N,"CHANGE IN ELBO:",elbo_change)
        elbo_score = np.append(elbo_score,elbo)
        #print("Finished Iteration " + str(iters_total) + ": ","change in Omega,B,Z",change_prec,change_B,change_z)
        
        adj_est = EZ > .5
        spars = np.mean(adj_est)/2

    print("v0 = ", v0, "Sparsity = ", spars,"ELBO = ",elbo)
    return (omega, EZ, phi,B,iters_total,elbo,elbo_score)



In [3]:
x = np.genfromtxt("OTU.csv")
m = np.genfromtxt("Cytokine.csv")

In [4]:
v0 = 0.01
v1 = 10
lamb = 150
vB = 1
a_gamma = 2
b_gamma = 2
a_pi = 2
b_pi = 2
max_iters = 40
tol_prec = 0.01
tol_elbo = 1
tol_B = 0.01
cpus = 1
omega, EZ, phi,B,iters_total, elbo, elbo_score  = SINC(x, m, v0, v1, lamb, vB,a_gamma,b_gamma,a_pi,b_pi, max_iters, tol_prec, tol_elbo, cpus)

('ELBO:', -64504.4579143266, 'CHANGE IN ELBO:', 10000)
('ELBO:', -57076.43987832983, 'CHANGE IN ELBO:', 7428.018035996771)
('ELBO:', -56352.36636931828, 'CHANGE IN ELBO:', 724.0735090115486)
('ELBO:', -55871.96813323125, 'CHANGE IN ELBO:', 480.3982360870359)
('ELBO:', -55489.039570231536, 'CHANGE IN ELBO:', 382.92856299971027)
('ELBO:', -55235.08923159586, 'CHANGE IN ELBO:', 253.9503386356737)
('ELBO:', -55040.39099917434, 'CHANGE IN ELBO:', 194.69823242152052)
('ELBO:', -54899.03432671014, 'CHANGE IN ELBO:', 141.35667246420053)
('ELBO:', -54789.10278827933, 'CHANGE IN ELBO:', 109.93153843081382)
('ELBO:', -54710.21283573626, 'CHANGE IN ELBO:', 78.88995254306792)
('ELBO:', -54647.91848687289, 'CHANGE IN ELBO:', 62.294348863368214)
('ELBO:', -54588.638095044145, 'CHANGE IN ELBO:', 59.28039182874636)
('ELBO:', -54517.178065252236, 'CHANGE IN ELBO:', 71.46002979190962)
('ELBO:', -54488.259186190124, 'CHANGE IN ELBO:', 28.918879062111955)
('ELBO:', -54460.218030477554, 'CHANGE IN ELBO:', 2

In [22]:
np.savetxt("Estimated_Precision.csv",omega, delimiter=",")
np.savetxt("Estimated_MicroMicro_Edge_Prob.csv",EZ, delimiter=",")
np.savetxt("Estimated_Coefficients.csv",B, delimiter=",")
np.savetxt("Estimated_CytoMicro_Edge_Prob.csv",phi, delimiter=",")

In [24]:
v0 = 0.01
v1 = 10
lamb = 150
vB = 1
a_gamma = 2
b_gamma = 2
a_pi = 2
b_pi = 2
max_iters = 50
tol_prec = 0.01
tol_elbo = 1
tol_B = 0.01
cpus = 1
omega, EZ, phi,B,iters_total, elbo, elbo_score  = SINC_fixed_Omega(x, m, v0, v1, lamb, vB,a_gamma,b_gamma,a_pi,b_pi, max_iters, tol_prec, tol_elbo, cpus)

(90, 3, 29)
(-31910.404914961357, -9526.920829688515, -44351.32463209643, -85788.65037674631, 85788.65037674631)
('Finished Iteration 1: ', 'change in Omega,B,Z', 0.0, 0.9557814934402833, 4.851179829522034)
(90, 3, 29)
(-16895.163287177405, -9526.920829688515, -44344.532495986125, -70766.61661285204, 15022.033763894273)
('Finished Iteration 2: ', 'change in Omega,B,Z', 0.0, 0.539547135969085, 1.237017300881842)
(90, 3, 29)
(-14367.928232024504, -9526.920829688515, -44340.15661734701, -68235.00567906004, 2531.610933792006)
('Finished Iteration 3: ', 'change in Omega,B,Z', 0.0, 0.05226030986781538, 0.5203641675067794)
(90, 3, 29)
(-13523.518510667855, -9526.920829688515, -44335.96416991076, -67386.40351026713, 848.6021687929024)
('Finished Iteration 4: ', 'change in Omega,B,Z', 0.0, 0.04577900478294184, 0.6237823313444393)
(90, 3, 29)
(-13344.949335265945, -9526.920829688515, -44331.50722054069, -67203.37738549514, 183.02612477199)
('Finished Iteration 5: ', 'change in Omega,B,Z', 0.0, 0

In [25]:
np.savetxt("Estimated_Precision_fixed_Omega.csv",omega, delimiter=",")
np.savetxt("Estimated_MicroMicro_Edge_Prob_fixed_Omega.csv",EZ, delimiter=",")
np.savetxt("Estimated_Coefficients_fixed_Omega.csv",B, delimiter=",")
np.savetxt("Estimated_CytoMicro_Edge_Prob_fixed_Omega.csv",phi, delimiter=",")

In [28]:
v0 = 0.01
v1 = 10
lamb = 150
vB = 1
a_gamma = 2
b_gamma = 2
a_pi = 2
b_pi = 2
max_iters = 50
tol_prec = 0.01
tol_elbo = 1
tol_B = 0.01
cpus = 1
omega, EZ, phi,B,iters_total, elbo, elbo_score  = SINC_fixed_B(x, m, v0, v1, lamb, vB,a_gamma,b_gamma,a_pi,b_pi, max_iters, tol_prec, tol_elbo, cpus)

(0, -19284.310187152965, -44193.72016420862, -63478.03035136159, 63478.03035136159)
('Finished Iteration 1: ', 'change in Omega,B,Z', 66.83899565941304, 0.0, 3.9914072154870532)
(0, -11801.198737947909, -44188.66999765817, -55989.868735606084, 7488.161615755504)
('Finished Iteration 2: ', 'change in Omega,B,Z', 0.12390842210407205, 0.0, 1.0932315549565677)
(0, -11251.000765361678, -44187.534859449144, -55438.53562481082, 551.3331107952617)
('Finished Iteration 3: ', 'change in Omega,B,Z', 0.1332889533125058, 0.0, 0.752286427347908)
(0, -11178.70869963215, -44186.947844746974, -55365.656544379126, 72.87908043169591)
('Finished Iteration 4: ', 'change in Omega,B,Z', 0.07899243449493643, 0.0, 0.6685294686990195)
(0, -11169.411545477271, -44186.49077901219, -55355.902324489456, 9.754219889669912)
('Finished Iteration 5: ', 'change in Omega,B,Z', 0.042276583615579066, 0.0, 0.5527262258626018)
(0, -11166.472768730942, -44186.133711957445, -55352.60648068839, 3.2958438010682585)
('Finished It

In [29]:
np.savetxt("Estimated_Precision_fixed_B.csv",omega, delimiter=",")
np.savetxt("Estimated_MicroMicro_Edge_Prob_fixed_B.csv",EZ, delimiter=",")
np.savetxt("Estimated_Coefficients_fixed_B.csv",B, delimiter=",")
np.savetxt("Estimated_CytoMicro_Edge_Prob_fixed_B.csv",phi, delimiter=",")